# Make MTH5 from legacy Phoenix MTU 

Older MTU units output data in a different format than the newer MTU-5C units.  The data are stored in `.TSN` files where the `N` represents the sample rate, these files are accompanied by a `.TBL` file which contains the metadata needed to characterize the data.  Inside each `.TSN` file is the data for all channels recorded at the sample rate.    

Thanks to Dong Hao for his Python code, a reader for the `.TBL` and `.TSN` files is now included in `mth5`. The reader output is a `RunTS` object.

<div class="alert alert-block alert-warning">
<b>TODO:</b> update `PhoenixCollection` to accommodate the older file format and then update `PhoenixCollection.assign_run_names` to reflect the older format and expected heirarchy. Then `MakeMTH5.from_phoenix` can be used. 
</div>  

<div class="alert alert-block alert-info">
<b>Note:</b> This example assumes that mth5_test_data is installed
</div>

## Imports

In [8]:

from pathlib import Path
from mth5_test_data import get_test_data_path
from mth5.io.phoenix import read_phoenix
from mth5.mth5 import MTH5


## Example Data

These data come courtesy of Dong Hao.  They reside in `mth5_test_data.phoenix_mtu`.  Lets load in the path and have a look at the files.

You will see there are 3 `TSN` files one for each sample rate, and one `TBL` file that provides the metadata information about the station recording.

In [4]:
mtu_path = get_test_data_path("phoenix_mtu")

In [6]:
ts_files = []
for fn in mtu_path.iterdir():
    print(fn.name)
    if fn.suffix.upper() in [".TS2", ".TS3", ".TS4", ".TS5", ".TSL", ".TSH"]:
        ts_files.append(fn)
    elif fn.suffix.upper() == ".TBL":
        mtu_table_filepath = fn

1690C16C.TBL
1690C16C.TS3
1690C16C.TS4
1690C16C.TS5
phoenix_mtu_test_data.zip


### Table File
The `TBL` file provides the metadata for the station recording.  This information has been crudely translated to `mt-metadata.timeseries` objects.  The metadata key names are up to 4 characters and are therefore difficult to translate without the proper documentation.

In [9]:
mtu_table = read_phoenix(mtu_table_filepath)

KeyError: "File extension 'TBL' is not supported by any Phoenix reader."